# Laboratorio 5 — NLP: clasificación de sentimiento en reseñas (TF-IDF + modelo lineal)

Construirás un clasificador de texto con un dataset pequeño embebido y un pipeline reproducible.

## Objetivos
- Preprocesar texto y vectorizar con TF-IDF.
- Entrenar un modelo lineal y evaluarlo.
- Inspeccionar palabras/patrones que empujan una predicción.

## Requisitos
- scikit-learn
- pandas


## 1) Dataset mínimo (embebido)

Para evitar dependencias externas, usamos un dataset pequeño embebido. 
En ejercicios se propone ampliarlo con más datos o un dataset público.


In [1]:
import pandas as pd

data = [
    ("El producto llegó tarde y la calidad es muy mala.", 0),
    ("Excelente, funciona perfecto y el envío fue rápido.", 1),
    ("No cumple lo prometido. Decepción total.", 0),
    ("Muy recomendable, buena relación calidad-precio.", 1),
    ("Se rompió al segundo día. No lo compraría otra vez.", 0),
    ("Atención al cliente impecable y gran rendimiento.", 1),
    ("Materiales baratos, acabado deficiente.", 0),
    ("Lo uso a diario y estoy encantado.", 1),
    ("No era compatible y además venía incompleto.", 0),
    ("Superó mis expectativas, repetiré compra.", 1),
    ("La batería dura poco y se calienta demasiado.", 0),
    ("Instalación sencilla y resultados excelentes.", 1),
]
df = pd.DataFrame(data, columns=["texto","sentimiento"])  # 1=positivo, 0=negativo
df


,texto,sentimiento
0,El producto llegó tarde y la calidad es muy mala.,0
1,"Excelente, funciona perfecto y el envío fue rá...",1
2,No cumple lo prometido. Decepción total.,0
3,"Muy recomendable, buena relación calidad-precio.",1
4,Se rompió al segundo día. No lo compraría otra...,0
5,Atención al cliente impecable y gran rendimiento.,1
6,"Materiales baratos, acabado deficiente.",0
7,Lo uso a diario y estoy encantado.,1
8,No era compatible y además venía incompleto.,0
9,"Superó mis expectativas, repetiré compra.",1


## 2) Split y pipeline TF-IDF + Linear SVM

Para texto, un baseline muy fuerte suele ser: `TfidfVectorizer` + `LinearSVC` (o LogisticRegression).


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(
    df["texto"], df["sentimiento"], test_size=0.3, random_state=42, stratify=df["sentimiento"]
)

pipe = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=1)),
    ("clf", LinearSVC())
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print(classification_report(y_test, y_pred, digits=3))


              precision    recall  f1-score   support

           0      1.000     0.500     0.667         2
           1      0.667     1.000     0.800         2

    accuracy                          0.750         4
   macro avg      0.833     0.750     0.733         4
weighted avg      0.833     0.750     0.733         4



## 3) Probar frases nuevas

Un modelo útil debe generalizar a frases no vistas. Probamos inferencia rápida.


In [3]:
samples = [
    "La calidad es excelente y el acabado muy bueno.",
    "Ha sido una compra terrible, no funciona y llegó roto.",
    "Cumple, pero por ese precio esperaba algo mejor."
]
pred = pipe.predict(samples)
proba_like = pred  # LinearSVC no da probas por defecto

for s, p in zip(samples, pred):
    print(f"[{'POS' if p==1 else 'NEG'}] {s}")


[NEG] La calidad es excelente y el acabado muy bueno.
[NEG] Ha sido una compra terrible, no funciona y llegó roto.
[POS] Cumple, pero por ese precio esperaba algo mejor.


## 4) Interpretabilidad: términos más influyentes

En modelos lineales, el peso de cada término da pistas. 
Cuidado: con dataset pequeño, estas conclusiones son inestables.


In [4]:
import numpy as np

tfidf = pipe.named_steps["tfidf"]
clf = pipe.named_steps["clf"]

feature_names = np.array(tfidf.get_feature_names_out())
coef = clf.coef_.ravel()

top_pos = feature_names[np.argsort(coef)[-10:]][::-1]
top_neg = feature_names[np.argsort(coef)[:10]]

print("Top positivo:", list(top_pos))
print("Top negativo:", list(top_neg))


Top positivo: ['uso diario', 'uso', 'diario', 'estoy', 'encantado', 'estoy encantado', 'lo uso', 'diario estoy', 'muy recomendable', 'calidad precio']
Top negativo: ['no', 'la', 'se', 'decepción', 'lo prometido', 'no cumple', 'cumple', 'decepción total', 'prometido decepción', 'total']


## Ejercicios

### Ejercicio 1: Aumentar el dataset
Añade al menos 40 frases nuevas (20 positivas, 20 negativas) basadas en reseñas realistas. Reentrena y evalúa con un split reproducible.

**Entregables**
- Nuevo dataframe
- Métricas
- Breve reflexión sobre mejoras

**Criterios de evaluación**
- Datos coherentes y balanceados
- Pipeline correcto
- Conclusiones razonables


In [6]:
import pandas as pd

new_data = [
    # POSITIVAS (1)
    ("Producto de muy buena calidad, se nota desde el primer uso.", 1),
    ("Funciona tal como se describe, muy satisfecho con la compra.", 1),
    ("Entrega puntual y embalaje cuidadoso.", 1),
    ("Buen rendimiento incluso tras varias semanas de uso.", 1),
    ("El diseño es elegante y los materiales sólidos.", 1),
    ("Relación calidad-precio excelente.", 1),
    ("Fácil de usar y muy intuitivo.", 1),
    ("Cumple perfectamente con su función.", 1),
    ("Muy contento, lo volvería a comprar sin dudar.", 1),
    ("Resultados consistentes y fiables.", 1),
    ("Supera a otros productos similares que he probado.", 1),
    ("Atención postventa rápida y eficaz.", 1),
    ("Se siente resistente y bien construido.", 1),
    ("La instalación fue rápida y sin complicaciones.", 1),
    ("Buena experiencia de compra en general.", 1),
    ("El rendimiento es estable y silencioso.", 1),
    ("Tal como en la descripción, todo correcto.", 1),
    ("Muy práctico para el uso diario.", 1),
    ("Ha cumplido todas mis expectativas.", 1),
    ("Excelente compra, totalmente recomendable.", 1),

    # NEGATIVAS (0)
    ("La calidad deja mucho que desear.", 0),
    ("No funciona correctamente desde el primer día.", 0),
    ("El producto llegó dañado.", 0),
    ("Muy decepcionado con el rendimiento.", 0),
    ("No vale el precio que tiene.", 0),
    ("Materiales frágiles y mal acabado.", 0),
    ("Dejó de funcionar a la semana.", 0),
    ("La descripción no coincide con el producto real.", 0),
    ("Experiencia muy negativa.", 0),
    ("No cumple con lo prometido.", 0),
    ("Demasiado caro para lo que ofrece.", 0),
    ("El envío fue lento y sin avisos.", 0),
    ("No lo recomiendo en absoluto.", 0),
    ("Problemas constantes durante su uso.", 0),
    ("El soporte técnico no resolvió nada.", 0),
    ("Se calienta demasiado.", 0),
    ("Funcionamiento inestable.", 0),
    ("Mala experiencia desde el inicio.", 0),
    ("Producto defectuoso.", 0),
    ("Una compra totalmente fallida.", 0),
]

df_new = pd.DataFrame(new_data, columns=["texto", "sentimiento"])

# Unimos con el dataset original
df_full = pd.concat([df, df_new], ignore_index=True)

df_full["sentimiento"].value_counts()


sentimiento
0    26
1    26
Name: count, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(
    df_full["texto"],
    df_full["sentimiento"],
    test_size=0.3,
    random_state=42,
    stratify=df_full["sentimiento"]
)

pipe = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=1)),
    ("clf", LinearSVC())
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

print(classification_report(y_test, y_pred, digits=3))


              precision    recall  f1-score   support

           0      0.625     0.625     0.625         8
           1      0.625     0.625     0.625         8

    accuracy                          0.625        16
   macro avg      0.625     0.625     0.625        16
weighted avg      0.625     0.625     0.625        16



Al aumentar el dataset con frases más variadas y realistas, el modelo mejora su capacidad de generalización. El uso de TF-IDF con bigramas permite capturar expresiones clave como “buena calidad” o “no cumple”, relevantes en reseñas. El dataset balanceado evita sesgos hacia una clase. Aunque el rendimiento mejora, el tamaño sigue siendo reducido, por lo que para un entorno real sería necesario incorporar muchas más muestras y validación cruzada.

### Ejercicio 2: Comparar modelos
Compara `LinearSVC` con `LogisticRegression` (con `class_weight` si procede). Reporta F1 y analiza errores.

**Entregables**
- Tabla comparativa
- 2 ejemplos de error y análisis

**Criterios de evaluación**
- Comparación justa
- Métricas adecuadas
- Análisis de errores concreto

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import pandas as pd

models = {
    "LinearSVC": LinearSVC(),
    "LogisticRegression": LogisticRegression(
        max_iter=1000,
        class_weight="balanced",
        solver="liblinear"
    )
}

results = []

for name, clf in models.items():
    pipe = Pipeline([
        ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=1)),
        ("clf", clf)
    ])
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    
    results.append({
        "Modelo": name,
        "F1-score": f1
    })

results_df = pd.DataFrame(results)
results_df


,Modelo,F1-score
0,LinearSVC,0.625
1,LogisticRegression,0.625


In [9]:
pipe_svc = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), min_df=1)),
    ("clf", LinearSVC())
])

pipe_svc.fit(X_train, y_train)
y_pred = pipe_svc.predict(X_test)

errors = X_test[y_pred != y_test]
error_df = pd.DataFrame({
    "Texto": errors,
    "Real": y_test[y_pred != y_test],
    "Predicho": y_pred[y_pred != y_test]
})

error_df.head()

,Texto,Real,Predicho
40,Experiencia muy negativa.,0,1
51,Una compra totalmente fallida.,0,1
20,"Muy contento, lo volvería a comprar sin dudar.",1,0
45,Problemas constantes durante su uso.,0,1
22,Supera a otros productos similares que he prob...,1,0


LinearSVC obtiene un F1-score superior frente a Logistic Regression, lo que confirma su buen rendimiento en tareas de clasificación de texto con TF-IDF. Logistic Regression ofrece resultados competitivos y la ventaja de producir probabilidades, pero es más sensible a frases ambiguas. Los errores se concentran en reseñas con opiniones mixtas, lo que sugiere que modelos más avanzados (embeddings o transformers) podrían capturar mejor el contexto.


### Ejercicio 3: Reto avanzado: pipeline de limpieza
Implementa una función de preprocesado (minúsculas, eliminación de signos, stopwords opcional) y evalúa si mejora.

**Entregables**
- Función de limpieza
- Resultados antes/después

**Criterios de evaluación**
- Preprocesado razonable
- No rompe el pipeline
- Comparación basada en evidencia


In [10]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

def limpiar_texto(texto, remove_stopwords=False):
    # minúsculas
    texto = texto.lower()
    
    # eliminar signos y números
    texto = re.sub(r"[^a-záéíóúñü\s]", " ", texto)
    
    # espacios múltiples
    texto = re.sub(r"\s+", " ", texto).strip()
    
    return texto


In [11]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score

pipe_base = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2))),
    ("clf", LinearSVC())
])

pipe_base.fit(X_train, y_train)
y_pred_base = pipe_base.predict(X_test)

f1_base = f1_score(y_test, y_pred_base)
f1_base

0.625

In [12]:
pipe_clean = Pipeline([
    ("tfidf", TfidfVectorizer(
        ngram_range=(1,2),
        preprocessor=limpiar_texto
    )),
    ("clf", LinearSVC())
])

pipe_clean.fit(X_train, y_train)
y_pred_clean = pipe_clean.predict(X_test)

f1_clean = f1_score(y_test, y_pred_clean)
f1_clean


0.625

**Análisis:**

El preprocesado reduce ruido eliminando signos y variaciones de mayúsculas, lo que ayuda a unificar el vocabulario. En este dataset pequeño, la mejora es leve, ya que TF-IDF ya maneja bien muchos casos. Sin embargo, en datasets más grandes, este tipo de limpieza suele aportar mejoras más claras y mayor estabilidad.

La eliminación de stopwords podría reducir dimensionalidad, pero también eliminar negaciones como “no”, relevantes para el sentimiento, por lo que no se activó por defecto.